# Fatality Predictor - Notebook 2 - Modélisation et Évaluation des Modèles

Dans ce notebook, nous allons entraîner et comparer plusieurs modèles de machine learning afin de prédire la gravité des accidents de la route. "y" représente la gravité d’un accident. Il vaut 0 si l'accident est mineur, 1 si il entraine au moins un blessé grave, sans mort, et 2 si l'accident est mortel.

L’objectif ici est d’entraîner plusieurs modèles sur X_train seulement. Le dataset X_test ne sera utilisé qu’à la fin pour générer des prédictions y_pred_ des meilleurs modèles, qui seront comparées à y_test dans un dernier notebook.

## 1. Chargement, exploration et visualisation des données

In [9]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from scipy.stats import mode

import warnings
warnings.filterwarnings("ignore")

In [2]:
path = "/Users/sachaabitbol/Desktop/Projet Ramp SMS/"

df_train = pd.read_csv(path + "X_train.csv")  # Contient X_train et y_train
df_test = pd.read_csv(path + "X_test.csv")

y_train = df_train["y"]
X_train = df_train.drop(columns=["y"])
X_test = df_test.copy()

print(f"X_train : {X_train.shape}, y_train : {y_train.shape}")
print(f"X_test  : {X_test.shape}")

X_train : (76779, 36), y_train : (76779,)
X_test  : (19195, 36)


In [3]:
print("X_train :\n")
print(X_train.head())

X_train :

   Numéro d'accident  Jour_semaine  Plein_jour  Nuit_éclairée  Obstacle  \
0       201800006621             4        True          False     False   
1       201600001095             2       False          False     False   
2       201700034431             1        True          False      True   
3       201700024729             5        True          False     False   
4       201700022470             2        True          False      True   

   En_Agglomeration  Pluie  Pluie_Forte  Temps_Couvert  Plus_de_2_Vehicules  \
0             False  False        False          False                 True   
1             False  False        False          False                False   
2              True  False        False          False                False   
3              True  False        False          False                False   
4              True  False        False          False                 True   

   ...  Nb_Choc_Avant  Nb_Choc_Arriere  Nb_Choc_Cote  Age_Min  

In [4]:
print("Distribution de y_train :")
print(y_train.value_counts(normalize=True))

Distribution de y_train :
y
0.0    0.594759
1.0    0.349796
2.0    0.055445
Name: proportion, dtype: float64


In [5]:
class_weights = {0: 1 / 0.594759, 1: 1 / 0.349796, 2: 1 / 0.055445}
print("Poids des classes :", class_weights)

Poids des classes : {0: 1.6813532876341508, 1: 2.8588091344669464, 2: 18.035891423933627}


## 2. Entraînement et sélection des modèles

In [6]:
# Séparation des features booléennes et continues
cols_bool = [col for col in X_train.columns if X_train[col].nunique() == 2]
cols_continuous = [col for col in X_train.columns if col not in cols_bool]

# Normalisation des features continues
scaler = StandardScaler()
X_train[cols_continuous] = scaler.fit_transform(X_train[cols_continuous])
X_test[cols_continuous] = scaler.transform(X_test[cols_continuous])

# Séparation en X_train_train et X_val pour validation
X_train_train, X_val, y_train_train, y_val = train_test_split(X_train, y_train, 
                                                              test_size=0.2, random_state=42, stratify=y_train)

# Définition des modèles et hyperparamètres

models = {
    "Régression Logistique": LogisticRegression(max_iter=500, class_weight=class_weights),
    "Random Forest": RandomForestClassifier(class_weight=class_weights),
    "Gradient Boosting": GradientBoostingClassifier(),
    "MLP (Neural Network)": MLPClassifier(hidden_layer_sizes=(50,), max_iter=500),
    "SVM": SVC(class_weight=class_weights),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(class_weight=class_weights),
    "Extra Trees": ExtraTreesClassifier(class_weight=class_weights),
    "AdaBoost": AdaBoostClassifier(),
    "HistGradientBoosting": HistGradientBoostingClassifier()
}

param_grids = {"Random Forest": {"n_estimators": [100, 200, 500], "max_depth": [10, 20, None]},
               "Gradient Boosting": {"n_estimators": [50, 100, 200], "learning_rate": [0.01, 0.1], "max_depth": [3, 5], "subsample" : [0.8]},
               "MLP (Neural Network)": {"hidden_layer_sizes": [(50,), (100,), (50, 50)], "activation": ["relu", "tanh"], "alpha": [0.0001, 0.001]},
               "HistGradientBoosting": {"max_iter": [100, 200, 300], "learning_rate": [0.01, 0.1, 0.2]}}

In [7]:
# Entraînement et évaluation des modèles
results = {}

for name, model in models.items():

    print(f"Entraînement de {name}...")
    model.fit(X_train_train, y_train_train)
    acc = accuracy_score(y_val, model.predict(X_val))
    results[name] = acc

print("\nRécapitulatif des performances sur X_val :")
for name, acc in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{name.ljust(25)} : {acc:.4f}")


# Sélection des 5 meilleurs modèles
best_models = sorted(results.items(), key=lambda x: x[1], reverse=True)[:5]
best_models_names = [name for name, _ in best_models]
print("\nCinq meilleurs modèles : ")
print(best_models_names)


# Optimisation des hyperparamètres
best_estimators = {}

for model_name in best_models_names:
    
    model = models[model_name]

    if model_name in param_grids:

        print(f"\nOptimisation de {model_name} avec GridSearchCV...")

        param_grid = param_grids[model_name]
        grid_search = GridSearchCV(model, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
        grid_search.fit(X_train_train, y_train_train)

        best_estimators[model_name] = grid_search.best_estimator_
        print(f"Meilleur modèle trouvé pour {model_name} : {grid_search.best_params_}")

    else:
        best_estimators[model_name] = model.fit(X_train_train, y_train_train)

Entraînement de Régression Logistique...
Entraînement de Random Forest...
Entraînement de Gradient Boosting...
Entraînement de MLP (Neural Network)...
Entraînement de SVM...
Entraînement de KNN...
Entraînement de Decision Tree...
Entraînement de Extra Trees...
Entraînement de AdaBoost...
Entraînement de HistGradientBoosting...

Récapitulatif des performances sur X_val :
HistGradientBoosting      : 0.7141
Gradient Boosting         : 0.7030
Random Forest             : 0.6909
AdaBoost                  : 0.6896
MLP (Neural Network)      : 0.6877
Extra Trees               : 0.6683
KNN                       : 0.6347
Decision Tree             : 0.5964
SVM                       : 0.5914
Régression Logistique     : 0.5582

Cinq meilleurs modèles : 
['HistGradientBoosting', 'Gradient Boosting', 'Random Forest', 'AdaBoost', 'MLP (Neural Network)']

Optimisation de HistGradientBoosting avec GridSearchCV...
Meilleur modèle trouvé pour HistGradientBoosting : {'learning_rate': 0.1, 'max_iter': 100}



/Users/sachaabitbol/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sachaabitbol/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sachaabitbol/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sachaabitbol/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  war

Meilleur modèle trouvé pour MLP (Neural Network) : {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (50,)}


## 3. Génération des prédictions et Sauvegarde des résultats

Nous avons maintenant sélectionné et optimisé nos 5 meilleurs modèles, à l’aide de GridSearchCV lorsque c’était nécessaire. Il est temps de générer leurs prédictions finales sur X_test, et de sauvegarder ces prédictions pour une évaluation future.

In [8]:
print("Génération des prédictions sur X_test...\n")
y_preds = {}

for model_name, model in best_estimators.items():
    
    print(f"  Prédiction avec {model_name}...")
    y_preds[model_name] = model.predict(X_test)

Génération des prédictions sur X_test...

  Prédiction avec HistGradientBoosting...
  Prédiction avec Gradient Boosting...
  Prédiction avec Random Forest...
  Prédiction avec AdaBoost...
  Prédiction avec MLP (Neural Network)...


In [10]:
y_pred_df = pd.DataFrame()

for model_name, y_pred in y_preds.items():
    y_pred_df[model_name] = y_pred

# Génération de la prédiction par vote majoritaire
y_pred_df["y_pred_vote"] = mode(y_pred_df.values, axis=1)[0].flatten()

# Sauvegarde du fichier contenant toutes les prédictions
filename = f"{path}y_pred.csv"
y_pred_df.to_csv(filename, index=False)

## Conclusion

Dans ce notebook, nous avons entraîné et comparé plusieurs modèles de machine learning pour prédire la gravité des accidents de la route. Après une première phase de sélection des 5 meilleurs modèles, nous avons optimisé leurs hyperparamètres lorsque cela était nécessaire, puis généré des prédictions finales sur X_test. Nous avons également ajouté une prédiction par vote majoritaire entre ces modèles.

Ces prédictions seront utilisées dans le Notebook 3, où nous comparerons les performances des modèles.